<a href="https://colab.research.google.com/github/casjunior93/Tradutor-de-Artigos-Tecnicos-com-AzureAI/blob/main/Tradu%C3%A7%C3%A3o_de_artigos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tradução de artigos

Desafio de projeto "Tradutor de Artigos Técnicos com AzureAI" da DIO. Faz parte do bootcamp "" da DIO em parceria com a Microsoft.

## Instalando bibliotecas

Bibliotecas de requisições HTTP e leitura de documentos docx.

In [1]:
!pip install requests python-docx

Importando bilbiotecas

In [2]:
import requests, uuid, json
from docx import Document

## Chaves do tradutor

In [3]:
CHAVE_API = "*****************************************************************"
URL_TRADUTOR = "https://api.cognitive.microsofttranslator.com/"
REGIAO = "eastus2"
TARGET_LANGUAGE = "pt-br"

## Funções de ajuda

In [4]:
# Tradução de texto
def traduzir_texto(texto, linguagem_objetivo):
  path = '/translate'
  constructed_url = URL_TRADUTOR + path

  params = {
    'api-version': '3.0',
    'from': 'en',
    'to': linguagem_objetivo #['fr', 'zu']
  }

  headers = {
    'Ocp-Apim-Subscription-Key': CHAVE_API,
    'Ocp-Apim-Subscription-Region': REGIAO,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
  }

  body = [{"text": texto}]

  request = requests.post(constructed_url, params=params, headers=headers, json=body)
  response = request.json()

  return response[0]['translations'][0]['text']

# Tradução de documento
def traduzir_documento(path, linguagem_objetivo, imprimir = True):
  documento = Document(path)
  texto_traduzido_string = ''
  texto_traduzido_lista = []

  for paragrafo in documento.paragraphs:
    texto_traduzido = traduzir_texto(paragrafo.text, linguagem_objetivo)
    texto_traduzido_string += texto_traduzido + '\n'
    texto_traduzido_lista.append(texto_traduzido)

  if imprimir:
    print(f'Tradução: {texto_traduzido_string}')

  documento_traduzido = Document()

  for linha in texto_traduzido_lista:
    documento_traduzido.add_paragraph(linha)

  path_traduzido = path.replace('.docx', f'_{linguagem_objetivo}.docx')
  print(f'Arquivo criado: {path_traduzido}')
  documento_traduzido.save(path_traduzido)

In [5]:
print(traduzir_texto('HI!', 'pt-br'))

OI!


## Criando um artigo em inglês usando o Gemini integrado ao Colab

Prompt: Crie um artigo básico em inglês sobre como configurar um roteador. Deve ter no máximo 5 parágrafos.

In [6]:
artigo_criado = """Configuring a router is the first step towards establishing a working internet connection and creating a secure network for your devices. While the process may seem daunting, it is actually quite straightforward with a few simple steps.

Start by connecting your router to your modem using an Ethernet cable. This creates the physical link between your router and the internet source. Next, connect your computer to the router either wirelessly using the router's default Wi-Fi network or with an Ethernet cable connected to one of the router's LAN ports. Once these physical connections are made, power on both your modem and your router.

Now, open a web browser on your computer and enter the router's default IP address into the address bar. This address, along with the default username and password, are usually found on a sticker located on the router or in its instruction manual. After entering the IP address, you'll be prompted to log in using these credentials.

Upon successful login, you'll be presented with the router's setup wizard or configuration page. This is where you will personalize your network and security settings. You'll need to enter information provided by your Internet Service Provider (ISP) such as the connection type and any necessary account details. Then, create a memorable name and a strong password for your Wi-Fi network. Make sure to enable security encryption to protect your data.

Finally, explore other available options, such as firewall settings and guest network features, to enhance your network's security and usability. Once you have finished configuring the desired settings, save the changes and reboot the router for them to take effect. Now you can enjoy the convenience and security of your personalized network!"""

Salvando em documento docx

In [7]:
documento_ingles = Document()

for paragrafo in artigo_criado.split('\n\n'):
  documento_ingles.add_paragraph(paragrafo)

documento_ingles.add_paragraph(artigo_criado)
documento_ingles.save('artigo_ingles.docx')

## Testando o tradutor de documento

In [8]:
traduzir_documento('artigo_ingles.docx', 'pt-br')

Tradução: Configurar um roteador é o primeiro passo para estabelecer uma conexão de internet funcional e criar uma rede segura para seus dispositivos. Embora o processo possa parecer assustador, na verdade é bastante simples com algumas etapas simples.
Comece conectando seu roteador ao modem usando um cabo Ethernet. Isso cria o link físico entre o roteador e a fonte da Internet. Em seguida, conecte seu computador ao roteador sem fio usando a rede Wi-Fi padrão do roteador ou com um cabo Ethernet conectado a uma das portas LAN do roteador. Depois que essas conexões físicas forem feitas, ligue o modem e o roteador.
Agora, abra um navegador da Web em seu computador e digite o endereço IP padrão do roteador na barra de endereços. Esse endereço, juntamente com o nome de usuário e a senha padrão, geralmente são encontrados em um adesivo localizado no roteador ou em seu manual de instruções. Depois de inserir o endereço IP, você será solicitado a fazer login usando essas credenciais.
Após o lo

# Testando a aplicação

## Instalando bibliotecas

In [9]:
!pip install openai beautifulsoup4 langchain-openai

In [10]:
import openai
from bs4 import BeautifulSoup

## Funções de  ajuda

In [11]:
def extrair_texto_url(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f'Erro ao acessar a URL: {response.status_code}')
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    article_tag = soup.find('article')
    if article_tag:
        p_tags = article_tag.find_all('p')
        texto = ""
        for p_tag in p_tags:
            texto += p_tag.get_text(separator=" ") + "\n"
    else:
        # Se a tag <article> não for encontrada, retorna None
        return None

    linhas = (linha.strip() for linha in texto.splitlines())
    texto_limpo = '\n'.join(linha for linha in linhas if linha)
    return texto_limpo

In [12]:
extrair_texto_url('https://dev.to/willvelida/building-your-network-in-the-cloud-a-beginners-guide-to-azure-virtual-networks-4h49')

"Posted on  Apr 30, 2023\nAs you look to move workloads to the cloud, it's important to have a secure and scalable network infrastructure that can support your applications. In Azure, this is where virtual networks come in.\nVirtual networks (VNet for short) are the fundamental building block for your private network in Azure. They enable many types of Azure resources to communicate with each other, the internet and on-prem networks securely. You can also use VNets to filter and route network traffic.\nIn this blog post, I'll start by talking about why we would use virtual networks and how we can plan our virtual networks to ensure they will support our architecture in Azure. I'll then show you how to create a basic virtual network using Bicep, before showing you how we can integrate Azure resources in a virtual network, using Azure Functions as an example.\nLet's dive in!\nAs I mentioned earlier, virtual networks enable Azure resources to communicate with other Azure resources, the in

## Usando Langchain

In [13]:
from langchain_openai.chat_models.azure import AzureChatOpenAI

In [14]:
client_azure_openai = AzureChatOpenAI(
    azure_endpoint = "https://dio-openai-eastus.openai.azure.com/",
    api_key = "*****************************************************************************",
    api_version = "2024-02-15-preview",
    deployment_name = "gpt-4o-mini",
    max_retries = 0
)

## Funções de ajuda

In [15]:
def traduzir_artigo(texto, linguagem):
  messages = [
      ("system", "Você atua como tradutor de textos"),
      ("user", f"Traduza o {texto} para o idioma {linguagem} e responda em markdown")
  ]

  response = client_azure_openai.invoke(messages)
  return response.content

## Testando:

In [16]:
print(f'Original: Start by connecting your router to your modem using an Ethernet cable.')
print(f"Traduzido: {traduzir_artigo('Start by connecting your router to your modem using an Ethernet cable.', 'pt-br')}")

Original: Start by connecting your router to your modem using an Ethernet cable.
Traduzido: Comece conectando seu roteador ao seu modem usando um cabo Ethernet.


## Testando com URL

In [17]:
url = "https://dev.to/willvelida/building-your-network-in-the-cloud-a-beginners-guide-to-azure-virtual-networks-4h49"

In [18]:
texto_ingles = extrair_texto_url(url)
texto_traduzido = traduzir_artigo(texto_ingles, 'pt-br')

Texto original:

In [19]:
print(texto_ingles)

Posted on  Apr 30, 2023
As you look to move workloads to the cloud, it's important to have a secure and scalable network infrastructure that can support your applications. In Azure, this is where virtual networks come in.
Virtual networks (VNet for short) are the fundamental building block for your private network in Azure. They enable many types of Azure resources to communicate with each other, the internet and on-prem networks securely. You can also use VNets to filter and route network traffic.
In this blog post, I'll start by talking about why we would use virtual networks and how we can plan our virtual networks to ensure they will support our architecture in Azure. I'll then show you how to create a basic virtual network using Bicep, before showing you how we can integrate Azure resources in a virtual network, using Azure Functions as an example.
Let's dive in!
As I mentioned earlier, virtual networks enable Azure resources to communicate with other Azure resources, the internet

Texto traduzido:

In [20]:
print(texto_traduzido)

### Publicado em 30 de abril de 2023

À medida que você busca mover cargas de trabalho para a nuvem, é importante ter uma infraestrutura de rede segura e escalável que possa suportar suas aplicações. No Azure, é aqui que as redes virtuais entram em cena.

As redes virtuais (VNet, para abreviar) são o bloco de construção fundamental para sua rede privada no Azure. Elas permitem que muitos tipos de recursos do Azure se comuniquem entre si, com a internet e com redes locais de forma segura. Você também pode usar VNets para filtrar e direcionar o tráfego da rede.

Neste post do blog, começarei falando sobre por que usar redes virtuais e como podemos planejar nossas redes virtuais para garantir que elas suportem nossa arquitetura no Azure. Em seguida, mostrarei como criar uma rede virtual básica usando Bicep, antes de demonstrar como podemos integrar recursos do Azure em uma rede virtual, usando Azure Functions como exemplo.

Vamos lá!

Como mencionei anteriormente, as redes virtuais permit